# Convergence Testing of the 2D Poisson Code

In [ ]:
using Plots; pyplot()
using LaTeXStrings
using Printf
using LinearAlgebra
using Revise

In [ ]:
default(xtickfont=font(14),  ytickfont=font(14), guidefont=font(14), 
    legendfont=font(12), lw=2,ms=8)

In [ ]:
using Poisson2D

## Convergence Problem
Use the eigenfunctions of $-\Delta$ on the box $(0,1)\times (0,1)$ with Dirichlet Boundary conditions,
$$
\varphi_{k_x,k_y}(x,y) = \sin(k_x \pi x) \sin(k_x \pi x).
$$
These have data
$$
f = -\Delta \varphi_{k_x,k_y} = \pi^2(k_x^2 + k_y^2) \sin(k_x \pi x) \sin(k_x \pi x)
$$
Any choice of $k_x$ and $k_y$ will do.

For convenience $n_x = n_y$ so $\Delta x = \Delta y$.

In [ ]:
a = 0;
b = 1;
c = 0;
d = 1;
n_vals = @. 10 * 2^(0:4)-1;

kx = 1;
ky = 1;
u_exact = (x,y) ->  sin(kx*π*(x-a)/(b-a)) * sin(ky*π*(y-c)/(d-c));
f = (x,y)->((kx*π/(b-a))^2 + (ky * π/(d-c))^2) * sin(kx*π*(x-a)/(b-a)) * sin(ky*π*(y-c)/(d-c))

In [ ]:
err_vals = Float64[];
for n in n_vals
    problem = FDPoisson2DProblem(a,b,c,d,n,f);
    assemble_system!(problem);
    U = direct_solve_poisson2d(problem);
    u = reshape(U,n,n);
    err = @. u - [u_exact(x_,y_) for x_ in problem.x, y_ in problem.y];
    push!(err_vals, norm(err,Inf))
end

In [ ]:
Δx_vals = @. 1 / (n_vals+1);

scatter(Δx_vals, err_vals,xscale=:log10, yscale=:log10, label="Error")
plot!(Δx_vals, Δx_vals.^2, label=L"$\propto \Delta x^2$")
xlabel!(L"$\Delta x$")